In [2]:
from geopy.distance import distance
import json
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Merge the Datasets into Travel Data

In [32]:
# merge place and location to get the lat/lon of each place

place = pd.read_csv('./data/Chicago_Data/MyDailyTravelData/place.csv', low_memory=False)
loc = pd.read_csv('./data/Chicago_Data/MyDailyTravelData/location.csv', low_memory=False)
pl = place[['sampno', 'perno', 'locno', 'arrtime', 
         'deptime', 'travtime', 'distance', 
         'mode','tpurp',
         'fare', 'plaza_total','pkamt']]
l = loc[['sampno', 'locno', 'latitude', 'longitude']]
pl_l = pd.merge(pl, l, left_on=['sampno', 'locno'], right_on=['sampno', 'locno'], how='left')
pl_l['arrtime']= pd.to_datetime(pl_l['arrtime'])
pl_l = pl_l.sort_values(by=['sampno', 'perno', 'arrtime'])

In [33]:
# merge person and household to get the social economic data of each person

pers = pd.read_csv('./data/Chicago_Data/MyDailyTravelData/person.csv', low_memory=False)
hh = pd.read_csv('./data/Chicago_Data/MyDailyTravelData/household.csv', low_memory=False)
p = pers.query('age > 0 and sex > 0 and educ > 0 and educ < 90')[['sampno', 'perno', 'age', 'sex','educ']]
h = hh.query('hhinc > 0')[['sampno', 'hhinc', 'hhveh','hhsize']]
pers_hh = pd.merge(p, h, left_on='sampno', right_on='sampno', how='inner')

In [34]:
# create travel data of each trips

trips = pd.merge(pers_hh, pl_l, left_on=['sampno', 'perno'], right_on=['sampno', 'perno'], how='right').rename(columns={'latitude': 'd_lat', 'longitude': 'd_lon'})
trips = trips.sort_values(by=['sampno', 'perno', 'arrtime']).reset_index(drop=True)

for i in range(1, len(trips)):
    if trips.loc[i-1, 'sampno'] == trips.loc[i, 'sampno'] and trips.loc[i-1, 'perno'] == trips.loc[i, 'perno']:
        trips.loc[i, 'o_lat'] = trips.loc[i-1, 'd_lat']
        trips.loc[i, 'o_lon'] = trips.loc[i-1, 'd_lon']
        

    else:
        trips.loc[i, 'o_lat'] = np.nan
        trips.loc[i, 'o_lon'] = np.nan


In [35]:
# count before filtering
initial_count = len(trips)
print(f"Total trips loaded: {initial_count}")

# filter out outliers
trips1 = trips.query(
    'travtime > 0 and travtime < 500 and '
    'distance > 0 and distance < 1000 and '
    'mode > 0 and mode < 900 and '
    'o_lat > 0 and d_lat > 0 and '
    'tpurp > 0'
).dropna(subset = ['age', 'sex', 'educ', 'hhinc', 'hhveh', 'hhsize'])

# count after filtering
filtered_count = len(trips1)
removed = initial_count - filtered_count
print(
    f"Trips after filtering: {filtered_count} "
    f"(removed {removed} invalid or outlier records)"
)

Total trips loaded: 128229
Trips after filtering: 97936 (removed 30293 invalid or outlier records)


In [36]:
# create unique OD pairs for google api 
df_for_google = trips1[['sampno', 'perno', 'age', 'sex', 'hhinc', 'hhveh','hhsize','educ', 'o_lat','o_lon','d_lat','d_lon']].dropna()
odpairs_for_google = df_for_google[['o_lat','o_lon','d_lat','d_lon']].drop_duplicates().reset_index(drop=True)
# odpairs_for_google.to_csv('./google_api_responses/odpairs_for_google.csv') # When we use the "gps_place.csv" file as the place table, we have 1.7w unique OD pairs
len(odpairs_for_google)

47118

## Read travel time from google api

In [37]:
# read in google api responses
google_df = pd.read_csv('./data/google_api_responses/combined_google_updated.csv')
google_df = google_df.apply(pd.to_numeric, errors='coerce')
google_df.describe()

,o_lat,o_lon,d_lat,d_lon,transit_time,walk_time,auto_time,bike_time,transit_dist,walk_dist,auto_dist,bike_dist,off_bus_time,in_bus_time,sum_parts
count,17645.000000,17645.000000,17645.000000,17645.000000,11227.000000,17559.000000,17645.000000,17645.000000,11227.000000,17559.000000,17645.000000,17645.000000,11227.000000,11227.000000,13247.000000
mean,41.878818,-87.899325,41.876877,-87.898878,78.865855,190.344945,17.132779,53.967527,20.159301,13.823465,16.003568,16.296274,46.573393,32.379652,66.913704
std,0.628517,1.339587,0.647076,1.349785,101.510344,458.886378,26.519831,158.759877,56.895594,33.721541,45.116110,50.719959,71.105894,51.291479,97.633240
min,27.775541,-122.623889,27.775541,-122.592739,1.483333,0.000000,0.000000,0.000000,0.411000,0.000000,0.000000,0.000000,0.000000,0.700000,0.000000
25%,41.790469,-88.066407,41.791126,-88.065576,29.766667,44.208333,7.400000,12.516667,4.845500,3.187000,3.546000,3.491000,16.116667,9.200000,19.766667
50%,41.902023,-87.812371,41.899737,-87.809725,50.150000,98.600000,12.716667,27.083333,10.189000,7.132000,7.889000,7.814000,26.016667,20.816667,42.316667
75%,42.005064,-87.671233,42.004414,-87.671107,87.250000,217.200000,21.616667,59.550000,22.590000,15.698000,17.803000,17.569000,48.058333,41.325000,77.208333
max,45.554763,-69.111946,45.551447,-69.620688,1753.600000,23650.400000,1019.866667,6365.000000,2112.300000,1751.552000,1852.219000,1997.228000,1634.950000,1491.366667,1753.600000


In [10]:
# merge google api responses with trips data
trips_with_mode_time = trips1.merge(google_df[['d_lon', 'd_lat', 'o_lon',
       'o_lat', 'transit_time', 'walk_time',
       'auto_time', 'bike_time', 'transit_dist', 'walk_dist', 'auto_dist',
       'bike_dist', 'off_bus_time', 'in_bus_time']], on=['o_lat','o_lon','d_lat','d_lon'], how='left')
trips_with_mode_time.describe()

,sampno,perno,age,sex,educ,hhinc,hhveh,hhsize,locno,arrtime,...,transit_time,walk_time,auto_time,bike_time,transit_dist,walk_dist,auto_dist,bike_dist,off_bus_time,in_bus_time
count,9.793600e+04,97936.000000,97936.000000,97936.000000,97936.000000,97936.000000,97936.000000,97936.000000,9.793600e+04,97936,...,11759.000000,38614.000000,38651.000000,38651.000000,11759.000000,38614.000000,38651.000000,38651.000000,11759.000000,11759.000000
mean,6.921336e+07,1.782674,39.338558,1.544335,4.110307,7.630820,1.860205,3.056108,6.340216e+05,2018-12-13 21:38:12.315757312,...,52.350485,62.576180,7.099714,17.308595,10.281506,4.548537,5.101141,5.158095,34.924917,17.536490
min,2.000008e+07,1.000000,5.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000e+04,2017-09-13 07:55:32,...,3.166667,0.000000,0.000000,0.000000,0.423000,0.000000,0.000000,0.000000,0.000000,0.733333
25%,7.001841e+07,1.000000,27.000000,1.000000,3.000000,7.000000,1.000000,2.000000,1.000000e+04,2018-09-24 17:54:00,...,19.700000,0.000000,0.000000,0.000000,2.597000,0.000000,0.000000,0.000000,12.516667,5.000000
50%,7.003528e+07,1.000000,39.000000,2.000000,5.000000,9.000000,2.000000,3.000000,3.000200e+04,2019-01-08 06:07:30,...,34.316667,32.950000,5.983333,9.733333,5.499000,2.375500,2.677000,2.580000,19.816667,9.383333
75%,7.004791e+07,2.000000,54.000000,2.000000,6.000000,9.000000,2.000000,4.000000,1.000003e+06,2019-02-28 14:58:35.249999872,...,56.325000,82.916667,10.266667,21.950000,10.771000,5.996000,6.579000,6.648000,37.483333,21.466667
max,7.010100e+07,12.000000,110.000000,2.000000,6.000000,10.000000,10.000000,12.000000,3.000007e+06,2019-05-10 02:34:00,...,1733.950000,3475.633333,664.350000,4404.600000,1371.038000,256.639000,1157.779000,1365.800000,1634.950000,1299.333333
std,5.842117e+06,1.053704,18.208231,0.498033,1.807179,2.491679,1.079166,1.468363,8.013257e+05,NaN,...,76.640623,102.812028,9.699944,47.644953,25.764599,7.496695,13.097551,14.747531,62.473579,29.202260


## Add travel cost

In [27]:
# place1 = pd.read_csv('./data/Chicago_Data/MyDailyTravelData/place.csv', low_memory=False)

# # merge the trips data with the place data and remove outliers
# df2 = pd.merge(trips1, place1[['sampno', 'perno', 'locno', 'pkamt']].drop_duplicates(subset=['sampno', 'perno', 'locno']),
#                left_on=['sampno', 'perno', 'locno'], right_on=['sampno', 'perno', 'locno'], how='left')

df2 = trips1.copy()
# transit cost
df2.loc[df2['fare'] < 0, 'fare'] = np.nan
df2.loc[df2['fare'] > 200, 'fare'] = np.nan

# Parking cost
df2.loc[df2['pkamt'] < 0, 'pkamt'] = np.nan
df2.loc[df2['pkamt'] > 200, 'pkamt'] = np.nan

# Toll cost
df2.loc[df2['plaza_total'] < 0, 'plaza_total'] = np.nan
df2.loc[df2['plaza_total'] > 200, 'plaza_total'] = np.nan

cost_df = (df2.groupby(['o_lat','o_lon','d_lat','d_lon'], as_index=False)
    .agg(
        transit_cost=('fare','mean'),
        parking_cost=('pkamt','mean'),
        toll_cost=('plaza_total','mean'),
    )
)
cost_df.describe()

,o_lat,o_lon,d_lat,d_lon,transit_cost,parking_cost,toll_cost
count,47118.000000,47118.000000,47118.000000,47118.000000,1740.000000,797.000000,4044.000000
mean,41.869638,-87.887912,41.869953,-87.886251,2.899035,21.190951,1.407740
std,0.548115,0.819305,0.551483,0.831557,2.614505,34.883007,0.885321
min,25.768504,-122.602418,25.768504,-122.602418,0.000000,0.000000,0.200000
25%,41.776190,-88.082054,41.776077,-88.082977,2.250000,2.833333,0.750000
50%,41.892196,-87.837312,41.892196,-87.837312,2.500000,9.000000,1.200000
75%,42.001982,-87.672522,42.001982,-87.672726,2.500000,18.000000,1.700000
max,47.328789,-71.007050,47.450481,-71.007050,60.000000,200.000000,8.870000


In [12]:
# merge cost table with trips data
trips_with_mode_time_cost = trips_with_mode_time.merge(cost_df, on=['o_lat','o_lon','d_lat','d_lon'], how='left')[['o_lat','o_lon','d_lat','d_lon','age', 'sex', 'educ', 'hhinc', 'hhveh', 'hhsize',
       'travtime', 'distance', 'mode',  'transit_time',
       'walk_time', 'auto_time', 'bike_time', 'transit_dist', 'walk_dist',
       'auto_dist', 'bike_dist', 'off_bus_time', 'in_bus_time', 'transit_cost',
       'parking_cost', 'toll_cost']]
trips_with_mode_time_cost.describe().T

,count,mean,std,min,25%,50%,75%,max
o_lat,97936.0,41.868141,0.452412,25.768504,41.750426,41.883393,42.006277,47.328789
o_lon,97936.0,-87.935769,0.678107,-122.602418,-88.157205,-87.916648,-87.681025,-71.007050
d_lat,97936.0,41.867862,0.459478,25.768504,41.750426,41.883491,42.006277,47.450481
d_lon,97936.0,-87.935348,0.683192,-122.602418,-88.157572,-87.916648,-87.681025,-71.007050
age,97936.0,39.338558,18.208231,5.000000,27.000000,39.000000,54.000000,110.000000
sex,97936.0,1.544335,0.498033,1.000000,1.000000,2.000000,2.000000,2.000000
educ,97936.0,4.110307,1.807179,1.000000,3.000000,5.000000,6.000000,6.000000
hhinc,97936.0,7.630820,2.491679,1.000000,7.000000,9.000000,9.000000,10.000000
hhveh,97936.0,1.860205,1.079166,0.000000,1.000000,2.000000,2.000000,10.000000
hhsize,97936.0,3.056108,1.468363,1.000000,2.000000,3.000000,4.000000,12.000000


## Save the data

In [13]:
# Define mode to main category mapping
mode_category_map = {
    101: "Walk",
    102: "Bike",
    103: "Bike",
    104: "Bike",
    202: "Auto",
    203: "Auto",
    301: "Auto",
    701: "Auto",
    702: "Auto",
    703: "Auto",
    704: "Auto",
    705: "Auto",
    401: "Public Transit",
    501: "Public Transit",
    502: "Public Transit",
    503: "Public Transit",
    504: "Public Transit",
    505: "Public Transit",
    506: "Public Transit",
    601: "Public Transit",
    201: "Other",
    801: "Other"
}

# Define mode to subcategory mapping
mode_subcategory_map = {
    101: "Walk",
    102: "Bike",
    103: "Bike",
    104: "Bike",
    202: "Private Vehicle",
    203: "Private Vehicle",
    301: "Taxi/Carpool",
    701: "Taxi/Carpool",
    702: "Rental",
    703: "Rental",
    704: "Taxi/Carpool",
    705: "Taxi/Carpool",
    401: "Bus",
    501: "Bus",
    502: "Bus",
    503: "Bus",
    504: "Bus",
    505: "transit/Metro",
    506: "transit/Metro",
    601: "Bus",
    201: "Other",
    801: "Other"
}
trips_with_mode_time_cost['mode_category'] = trips_with_mode_time_cost['mode'].map(mode_category_map)
trips_with_mode_time_cost['mode_subcategory'] = trips_with_mode_time_cost['mode'].map(mode_subcategory_map)

In [14]:
df3 = trips_with_mode_time_cost.rename(columns={'sex': 'male', 'hhveh': 'numvec'})
df3['highedu'] = df3['educ'].apply(lambda x: 1 if x >= 5 else 0)
df3 = df3.drop(columns=['educ'])
df3.to_csv('./MyDailyTravelData_original.csv', index=False)